In [1]:
import torch
import transformers
import datasets
import huggingface_hub

In [2]:
datasets.disable_caching()

In [3]:
cache_dir = '/Users/christopher/Documents/unirepsCache'
# cache_dir = '/net/scratch2/chriswolfram/hf_cache'

In [4]:
huggingface_hub.login(new_session=False)

In [5]:
model_name = 'openai-community/gpt2'
# model_name = 'meta-llama/Llama-3.2-1B'
# model_name = 'google/gemma-2-27b'

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, device_map='auto', cache_dir=cache_dir)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto', device_map='auto', cache_dir=cache_dir)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
# Add padding token if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [8]:
# TODO: Use split slicing and maybe shuffling+streaming to extract samples from large datasets
dataset = datasets.load_dataset('stanfordnlp/imdb', cache_dir=cache_dir)
dataset = dataset['test'].take(1024)

In [15]:
# TODO: Use split slicing and maybe shuffling+streaming to extract samples from large datasets
dataset = datasets.load_dataset('Skylion007/openwebtext', split='train', streaming=True, cache_dir=cache_dir, trust_remote_code=False)
# dataset = datasets.load_dataset('EleutherAI/the_pile_deduplicated', split='train', streaming=True, cache_dir=cache_dir, trust_remote_code=True)

In [16]:
datasets.Dataset.from_list(list(dataset.take(1000)))[0]

{'text': 'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing in pain and grasping at life, watched doctors and nurses walk away from a field hospital Friday night after a Belgian medical team evacuated the area, saying it was concerned about security.\n\nThe decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.\n\nCNN initially reported, based on conversations with some of the doctors, that the United Nations ordered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.\n\nHe said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical t

In [24]:
next(iter(dataset))

{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge.

In [12]:
model.get_output_embeddings().weight.detach()

tensor([[ 0.0045,  0.0166,  0.0210,  ..., -0.0054, -0.0422, -0.0315],
        [ 0.0215, -0.0238,  0.0211,  ..., -0.0107, -0.0011, -0.0374],
        [ 0.0136,  0.0104,  0.0128,  ...,  0.0081, -0.0122,  0.0051],
        ...,
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066],
        [ 0.0009,  0.0164, -0.0193,  ..., -0.0003, -0.0030,  0.0066]],
       device='mps:0', dtype=torch.bfloat16)

In [9]:
def compute_embeddings(example):
    tokens = tokenizer(example['text'], truncation=True, return_tensors='pt')
    input_ids = tokens['input_ids'].to(model.device)
    
    with torch.no_grad():
        model_out = model(input_ids=input_ids, output_hidden_states=True, use_cache=False)
    
    # The hidden states are cast to float32 becauase it avoids some numerical issues, and everything
    # will have to be cast eventually because Arrow doesn't currently support types like bfloat16.
    layer_token_embeddings = torch.stack(model_out.hidden_states)[:,0].float()

    # Outputs are moved to the CPU to avoid memory problems on the GPU.
    layer_last_embeddings = layer_token_embeddings[:,-1].cpu()
    layer_mean_embeddings = layer_token_embeddings.mean(1).cpu()

    return {
        'layer_last_embeddings': layer_last_embeddings,
        'layer_mean_embeddings': layer_mean_embeddings
    }

# TODO: This currently sets new_fingerprint because otherwise `map` appears to hash compute_embeddings which includes the entire model!
# This does not use batching. In experiments, batching (somehow) slightly slowed it down across models and hardware!
embeddings = dataset.take(128).map(compute_embeddings, new_fingerprint='test_fingerprint')
embeddings.set_format('torch')
# torch.cuda.empty_cache()

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1300 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
embeddings[-1]['layer_last_embeddings']

tensor([[ 5.1699e-02, -2.0490e-03,  7.3117e-03,  ..., -6.9872e-02,
          4.8656e-02,  9.6262e-02],
        [ 5.7095e-01, -1.3241e-01, -2.1094e-02,  ..., -8.0342e-01,
         -2.6857e-01, -8.2984e-02],
        [ 4.3036e-01, -1.5056e-01, -9.2372e-02,  ..., -9.6341e-01,
         -1.0211e-01,  1.9029e-01],
        ...,
        [ 8.2161e-01, -4.0093e+00,  3.6737e-01,  ..., -8.4385e-01,
         -9.6017e-01,  5.1732e+00],
        [ 1.6561e+00, -2.9995e+00, -6.4317e-01,  ...,  7.8280e-01,
         -3.0466e+00,  8.2880e+00],
        [ 8.4603e-02, -1.4935e-01,  1.8391e-02,  ...,  7.9412e-02,
         -2.6802e-01,  7.1571e-01]])

In [38]:
embeddings[-1]['layer_mean_embeddings']

tensor([[-2.0482e-01,  4.1662e-01, -1.7881e-02,  ...,  1.4470e-01,
          1.9332e-01, -5.1653e-02],
        [-1.2791e-01,  8.6794e-02,  1.1645e-02,  ...,  2.4013e-02,
          6.9477e-02, -1.4329e-01],
        [ 5.5292e-02, -1.2492e-01, -1.9515e-02,  ...,  7.1906e-03,
          4.3979e-02, -4.8419e-02],
        ...,
        [ 8.2082e+01,  6.4029e+01,  1.7222e+00,  ...,  2.3686e+01,
         -8.0630e+00, -7.2619e+01],
        [ 1.2726e+02,  8.3658e+01, -4.8099e+01,  ..., -3.4301e+00,
         -1.3376e+01, -1.1925e+02],
        [ 2.4463e+00,  2.1555e+00, -8.4142e-01,  ..., -1.2124e+00,
         -1.1269e+00, -6.6522e+00]])

In [18]:
with torch.no_grad():
    model_output = model(input_ids=tokenizer(embeddings['text'][-1], return_tensors='pt').input_ids.to(model.device), output_hidden_states=True, use_cache=False)

In [42]:
torch.stack(model_output.hidden_states)[:,0,-1]

tensor([[ 4.3045e-01,  5.2381e-01,  8.2463e-01,  ..., -1.1306e-01,
          3.3893e-02, -2.3120e-02],
        [-1.0142e-01,  2.8770e-01,  3.6419e-01,  ...,  6.0856e-02,
         -2.9304e-02, -8.4377e-02],
        [-3.7873e-02, -3.0839e-02,  3.2535e-01,  ...,  2.4453e-01,
         -9.8545e-02,  5.0027e-02],
        ...,
        [-5.3461e+01, -3.4155e+00, -2.3939e+02,  ...,  2.5938e+02,
          1.3480e+02, -1.9434e+02],
        [-3.0611e+02, -3.3730e+01, -4.7096e+02,  ...,  1.6534e+02,
          1.7137e+02, -4.1980e+02],
        [-9.2746e+00,  6.5281e-02, -1.0728e+01,  ...,  5.8639e+00,
          2.6707e+00, -1.5843e+01]], device='cuda:0')

In [34]:
torch.stack(model_output.hidden_states)[:,0].float().mean(1)

tensor([[-2.0482e-01,  4.1662e-01, -1.7881e-02,  ...,  1.4470e-01,
          1.9332e-01, -5.1653e-02],
        [-1.2791e-01,  8.6794e-02,  1.1645e-02,  ...,  2.4013e-02,
          6.9477e-02, -1.4329e-01],
        [ 5.5292e-02, -1.2492e-01, -1.9515e-02,  ...,  7.1906e-03,
          4.3979e-02, -4.8419e-02],
        ...,
        [ 8.2082e+01,  6.4029e+01,  1.7222e+00,  ...,  2.3686e+01,
         -8.0630e+00, -7.2619e+01],
        [ 1.2726e+02,  8.3658e+01, -4.8099e+01,  ..., -3.4301e+00,
         -1.3376e+01, -1.1925e+02],
        [ 2.4463e+00,  2.1555e+00, -8.4142e-01,  ..., -1.2124e+00,
         -1.1269e+00, -6.6522e+00]], device='cuda:0')

In [12]:
torch.stack(model_output.hidden_states).mean(2)[-1,0]

tensor([-1.5535,  1.9119,  0.9343,  ...,  1.7148, -1.6708,  1.1468],
       device='cuda:0')

In [98]:
(torch.stack(model_output.hidden_states).mean(2)[-1,0].cpu() - train_embeddings3['layer_mean_embeddings'][-1,-1]).abs().mean()

tensor(0.)

In [97]:
(torch.stack(model_output.hidden_states).mean(2)[-1,0].cpu() - train_embeddings2['layer_mean_embeddings'][-1,-1]).abs().mean()

tensor(0.)

In [75]:
(torch.stack(model_output.hidden_states).float().mean(2)[-1,0].cpu() - train_embeddings2['layer_mean_embeddings'][-1,-1]).abs().mean()

tensor(0.0019)

In [99]:
(torch.stack(model_output.hidden_states).float().mean(2)[-1,0].cpu() - train_embeddings3['layer_mean_embeddings'][-1,-1]).abs().mean()

tensor(0.0019)

In [141]:
input_layer_token_embeddings[0,-1,0]

tensor([ 1.5547, -1.5234,  2.7812,  ..., -0.6445,  0.6094, -1.0469],
       device='cuda:0', dtype=torch.bfloat16)

In [142]:
torch.stack(model_output.hidden_states).shape

torch.Size([17, 1, 98, 2048])

In [140]:
input_layer_token_embeddings = torch.stack(model_output.hidden_states).permute(1,0,2,3)

In [121]:
train_embeddings['layer_mean_embeddings'][-1] - torch.stack(model_output.hidden_states)[:,0].mean(1).cpu()

tensor([[ 0.0000e+00,  0.0000e+00, -6.1035e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  1.5259e-05,
          0.0000e+00,  0.0000e+00],
        [ 3.0518e-05,  1.5259e-05,  2.4414e-04,  ...,  0.0000e+00,
          0.0000e+00,  3.0518e-05],
        ...,
        [-9.7656e-04,  0.0000e+00,  4.8828e-04,  ..., -9.7656e-04,
         -4.8828e-04,  0.0000e+00],
        [ 0.0000e+00,  9.7656e-04, -1.2207e-04,  ...,  1.9531e-03,
         -1.2207e-03,  8.5449e-04],
        [-3.9062e-03,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  3.9062e-03]])

In [98]:
torch.stack(model_output.hidden_states).shape

torch.Size([17, 1, 161, 2048])

In [99]:
torch.stack(model_output.hidden_states)[:,0,-1]

tensor([[ 1.1414e-02, -2.2278e-03, -1.1292e-03,  ..., -1.7944e-02,
         -2.6855e-03, -2.1729e-02],
        [ 2.0142e-03,  4.8828e-03, -1.0400e-01,  ..., -2.1729e-02,
         -2.8809e-02, -3.7354e-02],
        [ 7.7515e-03, -5.6152e-02, -1.5430e-01,  ...,  5.4932e-02,
         -3.9551e-02, -6.4941e-02],
        ...,
        [-9.5703e-02,  3.2227e-01, -5.9766e-01,  ..., -7.6562e-01,
         -7.1289e-02, -5.2246e-02],
        [ 1.9141e-01,  4.3359e-01, -5.6641e-01,  ..., -6.3281e-01,
         -9.3262e-02, -6.0547e-02],
        [ 2.5312e+00,  3.9062e+00,  2.3730e-01,  ..., -4.7188e+00,
         -4.8438e+00, -1.0781e+00]], device='cuda:0', dtype=torch.bfloat16)

In [104]:
torch.stack(model_output.hidden_states)[:,0].mean(1)

tensor([[-9.9182e-04,  3.4027e-03,  1.2512e-02,  ..., -8.8501e-04,
         -6.8970e-03, -5.7068e-03],
        [-1.2085e-02,  8.4229e-03, -1.9653e-02,  ...,  9.2773e-03,
         -2.4048e-02,  4.2419e-03],
        [ 1.1292e-03,  3.9673e-03, -2.4658e-02,  ...,  1.7700e-02,
         -3.5400e-02, -3.6926e-03],
        ...,
        [ 3.4668e-02,  1.6309e-01, -1.3184e-01,  ..., -9.6680e-02,
         -1.6724e-02,  7.3730e-02],
        [ 9.0332e-02,  2.1484e-01, -7.2754e-02,  ..., -7.2754e-02,
         -1.3281e-01, -2.6611e-02],
        [ 1.3125e+00,  3.5312e+00,  6.1719e-01,  ..., -2.7969e+00,
         -3.2812e+00, -1.4648e-01]], device='cuda:0', dtype=torch.bfloat16)

In [8]:
def get_model_out(examples):
    tokens = tokenizer(examples['text'], padding='longest', return_tensors='pt')
    input_ids = tokens['input_ids'].to(model.device)
    attention_mask = tokens['attention_mask'].to(model.device)
    
    with torch.no_grad():
        model_out = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'model_out': model_out}

In [9]:
out = get_model_out(dataset.take(10))
torch.cuda.empty_cache()

In [11]:
input_ids = out['input_ids'].cpu()
attention_mask = out['attention_mask'].cpu()
model_out = out['model_out']

In [12]:
last_token_indices = attention_mask.sum(-1) - 1

In [23]:
torch.stack(model_out.hidden_states).cpu().permute(1,0,2,3).shape

torch.Size([10, 17, 452, 2048])

In [59]:
input_layer_token_embeddings = torch.stack(model_out.hidden_states).permute(1,0,2,3).cpu()

In [60]:
layer_last_embedding = input_layer_token_embeddings[torch.arange(input_layer_token_embeddings.shape[0]), :, last_token_indices]

In [42]:
input_layer_token_embeddings.shape

torch.Size([10, 17, 452, 2048])

In [50]:
attention_mask.shape

torch.Size([10, 452])

In [54]:
attention_mask.unsqueeze(0).shape

torch.Size([1, 10, 452])

In [73]:
attention_mask.unsqueeze(-1).unsqueeze(1).shape

torch.Size([10, 1, 452, 1])

In [80]:
attention_mask.sum(1)

tensor([315, 279, 146, 452, 160, 218, 361, 208, 177, 216])

In [83]:
((input_layer_token_embeddings * attention_mask.unsqueeze(-1).unsqueeze(1)).sum(2) / attention_mask.sum(1).unsqueeze(-1).unsqueeze(-1)).shape

torch.Size([10, 17, 2048])

In [84]:
layer_mean_embeddings = (input_layer_token_embeddings * attention_mask.unsqueeze(-1).unsqueeze(1)).sum(2) / attention_mask.sum(1).unsqueeze(-1).unsqueeze(-1)

In [85]:
layer_mean_embeddings

tensor([[[-1.9073e-03,  3.4180e-03,  1.1902e-02,  ..., -5.9814e-03,
          -5.7068e-03, -4.3640e-03],
         [-1.2329e-02,  1.1475e-02, -1.5625e-02,  ...,  4.3640e-03,
          -1.2085e-02,  1.6098e-03],
         [ 1.9531e-03,  6.3782e-03, -1.8555e-02,  ...,  1.0010e-02,
          -1.8433e-02, -5.5847e-03],
         ...,
         [-1.3086e-01, -3.5156e-02, -4.4861e-03,  ..., -1.4258e-01,
           3.2959e-02,  9.8633e-02],
         [-1.3965e-01,  3.3691e-02,  1.0840e-01,  ..., -1.3867e-01,
           2.4261e-03,  1.0107e-01],
         [-4.1406e-01,  2.4688e+00,  1.1172e+00,  ..., -2.7031e+00,
          -2.7656e+00,  7.1094e-01]],

        [[-2.7924e-03,  2.7618e-03,  1.0254e-02,  ..., -3.5400e-03,
          -4.1809e-03, -3.6621e-03],
         [-1.1230e-02,  1.0254e-02, -2.5635e-02,  ...,  3.9062e-03,
          -2.1729e-02,  2.5177e-03],
         [ 4.8065e-04,  8.8501e-03, -3.1006e-02,  ...,  1.0986e-02,
          -4.1504e-02, -8.0566e-03],
         ...,
         [ 4.1992e-02,  9

In [ ]:
(input_layer_token_embeddings.permute(1,0,2,3) * attention_mask.unsqueeze(-1)).mean().shape

torch.Size([17, 10, 452, 2048])

In [127]:
layer_mean_embeddings = ((torch.stack(model_out.hidden_states)[:, torch.arange(input_ids.size(0))] * attention_mask.unsqueeze(-1)).sum(2) / attention_mask.sum(-1).unsqueeze(-1)).permute(1,0,2)

torch.Size([10, 17, 2048])